In [1]:
import os
import pickle
import gc

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import L1L2

from sklearn.model_selection import train_test_split

In [2]:
MODEL = "7-auto-regressive-lstm-covariates"
COVARIATE_COLUMNS = ['covs_week_sin','covs_week_cos','covs_day_sin','covs_day_cos','covs_hour_sin','covs_hour_cos','covs_rbf_month_1','covs_rbf_month_2','covs_rbf_month_3','covs_rbf_month_4','covs_rbf_month_5','covs_rbf_month_6','covs_rbf_month_7','covs_rbf_month_8','covs_rbf_month_9','covs_rbf_month_10','covs_rbf_month_11','covs_rbf_month_12']

config = {
    "N_PREV": 120,
    "N_FORWARD": 48,
    "STEP_SIZE": 1,
    
    "LATENT_SIZE": 16,
    
    "LR": 0.001,
    "BATCH_SIZE": 256,
    "EPOCHS": 2,
    
    "L1": 0.0000001, 
    "L2": 0.0000001,
    
    "TARGET_COL": 'normalized_level',
}

In [3]:
import wandb
from wandb.keras import WandbCallback
from secrets import WANDB
wandb.login(key=WANDB)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\KiernanMcGuigan/.netrc


True

## **Load Data Function**

In [4]:
def get_samples(df, set_number):
    subset = df.loc[df.set==set_number, :]
    shape = subset.shape
    if(shape[0] > 1_000):
        subset = subset.iloc[-1_000:, :]
    data = subset[config['TARGET_COL']].values
    covariates = subset[COVARIATE_COLUMNS].values
    print(f'Data Shape: {shape}, Reduced Shape {data.shape}')
    
    X, y, past_covs, future_covs, dates = [], [], [], [], []
    for start_idx in range(0, data.shape[0]-config["N_PREV"]-config["N_FORWARD"], config["STEP_SIZE"]):
        mid_idx = start_idx + config["N_PREV"]
        end_idx = mid_idx + config["N_FORWARD"]
        X.append(data[start_idx:mid_idx])
        y.append(data[mid_idx:end_idx])
        past_covs.append(covariates[start_idx+1:mid_idx+1])
        future_covs.append(covariates[mid_idx+1:end_idx])
        dates.append(subset.iloc[mid_idx:end_idx].index.values)
    return np.array(X), np.array(y), np.array(past_covs), np.array(future_covs), np.array(dates)

def to_dataset(X1, X2, X3, y):
    dataset = tf.data.Dataset.from_tensor_slices(({'series':X1, 'past_dates':X2, 'future_dates':X3}, y))
    dataset = dataset.cache().shuffle(X1.shape[0]+1).batch(config["BATCH_SIZE"]).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def get_train_and_val_datasets(X1, X2, X3, y):
    X1_train, X1_val, X2_train, X2_val, X3_train, X3_val, y_train, y_val = train_test_split(X1, X2, X3, y, test_size=0.15, shuffle=True)
    train_ds = to_dataset(X1_train, X2_train, X3_train, y_train)
    val_ds = to_dataset(X1_val, X2_val, X3_val, y_val)
    return train_ds, val_ds

def df_to_samples(df):
    X, y, past, future, _ = get_samples(df, 0)
    train_ds, val_ds = get_train_and_val_datasets(X, past, future, y)
    X_test, y_test, past_test, future_test, dates = get_samples(df, 1)
    return train_ds, val_ds, X_test, y_test, past_test, future_test, dates
    
def get_data(fold):
    df = pd.read_csv(f'data/fold{fold}_data_v3.csv')
    with open(f'./data/fold{fold}_normalizer_v3.pkl', mode='rb') as f:
        normalizer = pickle.load(f)
    return df_to_samples(df), normalizer

## **Model Functions**

In [5]:
def get_model():
    inputs = tf.keras.layers.Input(shape=(config["N_PREV"], 1), name='series')
    past_covs = tf.keras.layers.Input(shape=(config["N_PREV"], len(COVARIATE_COLUMNS)), name='past_dates')
    future_covs = tf.keras.layers.Input(shape=(config["N_FORWARD"]-1, len(COVARIATE_COLUMNS)), name='future_dates')
    
    lstm1 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=True, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    lstm2 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=True, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    lstm3 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=False, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    dense = tf.keras.layers.Dense(1)
    
    predictions = []
    x = K.concatenate([inputs, past_covs], axis=-1)
    x, hidden_state1, cell_state1 = lstm1(x)
    x, hidden_state2, cell_state2 = lstm2(x)
    x, hidden_state3, cell_state3 = lstm3(x)
    x = dense(x)
    
    predictions.append(x)
    for idx in range(1, config['N_FORWARD']):
        x = tf.expand_dims(x, axis=1)
        x = K.concatenate([x, future_covs[:, idx-1:idx, :]])
        x, hidden_state1, cell_state1 = lstm1(x, initial_state=[hidden_state1, cell_state1])
        x, hidden_state2, cell_state2 = lstm2(x, initial_state=[hidden_state2, cell_state2])
        x, hidden_state3, cell_state3 = lstm3(x, initial_state=[hidden_state3, cell_state3])
        x = dense(x)
        
        predictions.append(x)
        
    outputs = tf.keras.layers.Lambda(lambda x: K.concatenate(x, axis=1))(predictions)
    model = tf.keras.models.Model(inputs=[inputs,past_covs,future_covs], outputs=outputs)
    model.compile(loss="mse", metrics=["mae"], optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"]))
    return model

# def get_model():
#     model = MyModel()
#     model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(config["LR"]), metrics=['mae'])
#     model.predict(tf.ones(shape=(config['BATCH_SIZE'],config['N_PREV'],1)))
#     return model
    
test_model = get_model()
print(test_model.summary())
del test_model
gc.collect()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 series (InputLayer)            [(None, 120, 1)]     0           []                               
                                                                                                  
 past_dates (InputLayer)        [(None, 120, 18)]    0           []                               
                                                                                                  
 tf.concat (TFOpLambda)         (None, 120, 19)      0           ['series[0][0]',                 
                                                                  'past_dates[0][0]']             
                                                                                                  
 lstm (LSTM)                    multiple             2304        ['tf.concat[0][0]',          

                                                                  'tf.concat_25[0][0]',           
                                                                  'lstm[24][1]',                  
                                                                  'lstm[24][2]',                  
                                                                  'tf.concat_26[0][0]',           
                                                                  'lstm[25][1]',                  
                                                                  'lstm[25][2]',                  
                                                                  'tf.concat_27[0][0]',           
                                                                  'lstm[26][1]',                  
                                                                  'lstm[26][2]',                  
                                                                  'tf.concat_28[0][0]',           
          

                                                                  'lstm[5][0]',                   
                                                                  'lstm_1[4][1]',                 
                                                                  'lstm_1[4][2]',                 
                                                                  'lstm[6][0]',                   
                                                                  'lstm_1[5][1]',                 
                                                                  'lstm_1[5][2]',                 
                                                                  'lstm[7][0]',                   
                                                                  'lstm_1[6][1]',                 
                                                                  'lstm_1[6][2]',                 
                                                                  'lstm[8][0]',                   
          

                                                                  'lstm_1[31][2]',                
                                                                  'lstm[33][0]',                  
                                                                  'lstm_1[32][1]',                
                                                                  'lstm_1[32][2]',                
                                                                  'lstm[34][0]',                  
                                                                  'lstm_1[33][1]',                
                                                                  'lstm_1[33][2]',                
                                                                  'lstm[35][0]',                  
                                                                  'lstm_1[34][1]',                
                                                                  'lstm_1[34][2]',                
          

                                                                  'lstm_2[11][2]',                
                                                                  'lstm_1[13][0]',                
                                                                  'lstm_2[12][1]',                
                                                                  'lstm_2[12][2]',                
                                                                  'lstm_1[14][0]',                
                                                                  'lstm_2[13][1]',                
                                                                  'lstm_2[13][2]',                
                                                                  'lstm_1[15][0]',                
                                                                  'lstm_2[14][1]',                
                                                                  'lstm_2[14][2]',                
          

...

 licingOpLambda)                                                                                  
                                                                                                  
 tf.concat_20 (TFOpLambda)      (None, 1, 19)        0           ['tf.expand_dims_19[0][0]',      
                                                                  'tf.__operators__.getitem_19[0][
                                                                 0]']                             
                                                                                                  
 tf.expand_dims_20 (TFOpLambda)  (None, 1, 1)        0           ['dense[20][0]']                 
                                                                                                  
 tf.__operators__.getitem_20 (S  (None, 1, 18)       0           ['future_dates[0][0]']           
 licingOpLambda)                                                                                  
          

 tf.concat_29 (TFOpLambda)      (None, 1, 19)        0           ['tf.expand_dims_28[0][0]',      
                                                                  'tf.__operators__.getitem_28[0][
                                                                 0]']                             
                                                                                                  
 tf.expand_dims_29 (TFOpLambda)  (None, 1, 1)        0           ['dense[29][0]']                 
                                                                                                  
 tf.__operators__.getitem_29 (S  (None, 1, 18)       0           ['future_dates[0][0]']           
 licingOpLambda)                                                                                  
                                                                                                  
 tf.concat_30 (TFOpLambda)      (None, 1, 19)        0           ['tf.expand_dims_29[0][0]',      
          

                                                                 0]']                             
                                                                                                  
 tf.expand_dims_38 (TFOpLambda)  (None, 1, 1)        0           ['dense[38][0]']                 
                                                                                                  
 tf.__operators__.getitem_38 (S  (None, 1, 18)       0           ['future_dates[0][0]']           
 licingOpLambda)                                                                                  
                                                                                                  
 tf.concat_39 (TFOpLambda)      (None, 1, 19)        0           ['tf.expand_dims_38[0][0]',      
                                                                  'tf.__operators__.getitem_38[0][
                                                                 0]']                             
          

 lambda (Lambda)                (None, 48)           0           ['dense[0][0]',                  
                                                                  'dense[1][0]',                  
                                                                  'dense[2][0]',                  
                                                                  'dense[3][0]',                  
                                                                  'dense[4][0]',                  
                                                                  'dense[5][0]',                  
                                                                  'dense[6][0]',                  
                                                                  'dense[7][0]',                  
                                                                  'dense[8][0]',                  
                                                                  'dense[9][0]',                  
          

513043

## **Train Loop**

In [6]:
def train_model(model, train_ds, val_ds):
    reducer = tf.keras.callbacks.ReduceLROnPlateau(monior='val_loss', factor=0.1, patience=2, mode='min', cooldown=1)
    stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, mode='min', restore_best_weights=True)
    model.fit(train_ds,
              epochs=config["EPOCHS"], 
              callbacks=[reducer, stopper, WandbCallback()],
              validation_data=val_ds)
    return model

def test_model(model, X_test, y_test, past_test, future_test, normalizer):
    predictions = model.predict({'series':X_test, 'past_dates':past_test, 'future_dates':future_test})
    inversed_predictions, inversed_actuals = [], []
    for pred, actual in zip(predictions, y_test):
        inversed_predictions.append(normalizer.inverse_transform(np.array(pred).reshape(-1,1)).reshape((-1)))
        inversed_actuals.append(normalizer.inverse_transform(np.array(actual).reshape(-1,1)).reshape((-1)))
    inversed_predictions = np.array(inversed_predictions)
    inversed_actuals = np.array(inversed_actuals)
    forward_error = np.mean(np.abs(inversed_actuals - inversed_predictions), axis=-1)
    error = np.mean(forward_error)
    return error, forward_error, inversed_predictions, inversed_actuals

def record_forward_error(forward_errors, error, run):
    run.log({'test/error': error})
    run.log({'test/forward_error': wandb.plot.line_series(
        xs=[i+1 for i in range(forward_errors.shape[0])],
        ys=[forward_errors],
        keys=['Forward Horizon Errors'],
        title=f'Forward Horizon Errors (MAE: {error})',
    )})
        
def graph_predictions(predictions, actuals, dates, run):
    for look_ahead in [1, 12, 24, 36, 48]:
        preds, true, time = predictions[:, look_ahead-1], actuals[:, look_ahead-1], dates[:, look_ahead-1]
        error = np.mean(np.abs(true - preds))
        run.log({f'test/forecast-{look_ahead}-hours-ahead': wandb.plot.line_series(
            xs=time,
            ys=[true, preds],
            keys=['Actuals', 'Predictions'],
            title=f'{look_ahead} Hour(s) Ahead Forecast (MAE: {error})',
            xname='week'
        )})

In [7]:
def train():
    for fold in range(1, 4):
        run = wandb.init(project="time-series-methods", entity="kmcguigan", group=f"{MODEL}-model", config=config, job_type="train")
        run.name = f'{MODEL}-fold-{fold}'
        (train_ds, val_ds, X_test, y_test, past_test, future_test, dates), normalizer = get_data(fold)
        model = get_model()
        model = train_model(model, train_ds, val_ds)
        error, forward_errors, predictions, actuals = test_model(model, X_test, y_test, past_test, future_test, normalizer)
        run.log({'test-error': error})
        record_forward_error(forward_errors, error, run)
        graph_predictions(predictions, actuals, dates, run)
        run.finish()
        del model
        del train_ds
        del val_ds
        gc.collect()
    return

In [8]:
train()

wandb: Currently logged in as: kmcguigan. Use `wandb login --relogin` to force relogin


Data Shape: (159096, 23), Reduced Shape (1000,)
Data Shape: (43390, 23), Reduced Shape (1000,)


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/2
3/3 [==============================] - 294s 30s/step - loss: 1.0235 - mae: 0.8246 - val_loss: 1.0191 - val_mae: 0.8192 - lr: 0.0010 - _timestamp: 1654479105.0000 - _runtime: 337.0000
Epoch 2/2
26/26 [==============================] - 61s 93ms/step


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


...

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁█
loss,█▁
lr,▁▁
mae,█▁
test-error,▁
test/error,▁
val_loss,█▁
val_mae,▁█
best_epoch,1
best_val_loss,0.9114
epoch,1


Data Shape: (245876, 23), Reduced Shape (1000,)
Data Shape: (43390, 23), Reduced Shape (1000,)
Epoch 1/2
3/3 [==============================] - 292s 30s/step - loss: 1.1094 - mae: 0.8299 - val_loss: 1.0205 - val_mae: 0.8046 - lr: 0.0010 - _timestamp: 1654480011.0000 - _runtime: 335.0000
Epoch 2/2
26/26 [==============================] - 59s 108ms/step


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁█
loss,█▁
lr,▁▁
mae,█▁
test-error,▁
test/error,▁
val_loss,█▁
val_mae,█▁
best_epoch,1
best_val_loss,0.94348
epoch,1
